In [ ]:
# importing required packages for building classification machine learning model
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.tree import export_graphviz
import six
import pydot
from sklearn import tree
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score, confusion_matrix #to check the accuracy of the model

In [ ]:
# reading forest types datasets which is downloaded from UCI
# https://archive.ics.uci.edu/ml/datasets/Forest+type+mapping
# download the data and upload it to Kaggle by clicking "Add Data"
df_train = pd.read_csv('../input/training.csv')
df_Test = pd.read_csv('../input/testing.csv')

In [ ]:
# displaying top 5 records to check whether its reading properly from kaggle server
# here the first column 'class' is the output and rest of the columns are inputs
# so we need to split the datasets into two
# 1 dataset contains only output, (i.e) column 'class'
# 2 dataset will have rest of the columns
df_train.head()

In [ ]:
df_train['class'].unique()
# Class: 's' ('Sugi' forest), 'h' ('Hinoki' forest), 'd' ('Mixed deciduous' forest), 'o' ('Other' non-forest land)

In [ ]:
X_train = df_train.iloc[:, 1:].values # extracting inputs from training dataset - column 1 to till end
y_train = df_train.iloc[:, 0].values # extracting output from training dataset - column 0

X_test = df_Test.iloc[:, 1:].values # extracting inputs from testing dataset - column 1 to till end
y_test = df_Test.iloc[:, 0].values # extracting output from training dataset - column 0

In [ ]:
# Feature Scaling - Our dataset values are not scaled, (i.e) columns values are not in specific range
# By applying feature scaling - the sum of values in all columns will be zero
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Implementing through Logistic Regression Algorithm which has highest accuracy - 86%**

In [ ]:
# implementing through logistic regression algorithm with penalty l1 and liblinear solver - 86%
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, penalty ='l1',solver='liblinear',
                                multi_class='ovr', C=6)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

In [ ]:
# displaying confusion matrix for calculating accuracy
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
# implementing through random forest algorithm
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

In [ ]:
# to check feature importance
feature_imp = pd.Series(classifier.feature_importances_, index= df_train.columns[1:]).sort_values(ascending=False)
sns.barplot(x=feature_imp, y =feature_imp.index)
plt.xlabel('Feature importance Score')
plt.ylabel('Features')
plt.title('Visualizing Important Features')
plt.legend(handles=[])
plt.show()

In [ ]:
# Saving inner decision trees generated by random forest algorithm
col = df_train.columns[1:]
dotfile = six.StringIO()
i_tree = 1
for tree_in_forest in classifier.estimators_:
    export_graphviz(tree_in_forest,out_file='tree.dot',
    feature_names=col,
    filled=True,
    rounded=True)
    (graph,) = pydot.graph_from_dot_file('tree.dot')
    name = 'tree' + str(i_tree)
    graph.write_png(name+  '.png')
    os.system('dot -Tpng tree.dot -o tree.png')
    i_tree +=1

In [ ]:
# Showing inner decision trees generated by random forest algorithm
fig=plt.figure(figsize=(50, 50), dpi=150, facecolor='w', edgecolor='k')
columns = 2
rows = 5
for i in range(1, 2):
    img = mpimg.imread('tree' + str(i) + '.png')    
    fig.add_subplot(rows, columns, i)
    plt.title('Random Forest Decision Tree' + str(i))
    plt.imshow(img)
plt.show()

**Implementing through other algorithm which has accuracy less than random forest**

In [ ]:
# implementing through naive bayes algorithm - 80%
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

In [ ]:
# implementing through XGB classifier algorithm - 78%
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)

In [ ]:
# implementing through decision tree algorithm - 76%
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred)*100)